This code downloads radar data via Amazon Web Services, then creates a 4-panel plot using Py-Art and Cartopy. Code for AWS retrieval is originally from Scott Collis: https://github.com/scollis  The code also makes use of gatefilters, and plots shape files. 

In [1]:
##Run this cell first
import warnings
warnings.filterwarnings('ignore')
from boto.s3.connection import S3Connection
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import cartopy.feature as cfeature
import gzip
from matplotlib import pyplot as plt
import shutil, os
from datetime import timedelta, datetime
import numpy as np
import tempfile
import pyart
def nearestDate(dates, pivot):
    return min(dates, key=lambda x: abs(x - pivot))


def get_radar_from_aws(site, datetime_t):
    """
    Get the closest volume of NEXRAD data to a particular datetime.
    Parameters
    ----------
    site : string
        four letter radar designation 
    datetime_t : datetime
        desired date time
    """
    
    #First create the query string for the bucket knowing
    #how NOAA and AWS store the data
    
    my_pref = datetime_t.strftime('%Y/%m/%d/') + site
    
    #Connect to the bucket
    
    conn = S3Connection(anon = True)
    bucket = conn.get_bucket('noaa-nexrad-level2')
    
    #Get a list of files 
    
    bucket_list = list(bucket.list(prefix = my_pref))
    #print(bucket_list)
    #we are going to create a list of keys and datetimes to allow easy searching
    #print bucket_list
    keys = []
    datetimes = []
    
    #populate the list

    for i in range(len(bucket_list)):
        this_str = str(bucket_list[i].key)
        if 'gz' in this_str:
            endme = this_str[-22:-3]
            fmt = '%Y%m%d_%H%M%S_V06' 
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])
            #print(dt)
        if this_str[-3::] == 'V06': #'LSX20160707_000150_' does not match format '%Y%m%d_%H%M%S_V06'
            #print(this_str)
            #print(this_str[-19::])
            endme = this_str[-19::]
            fmt = '%Y%m%d_%H%M%S_V06' 
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])
    
    #function to allow easy searching 
    
    def func(x):
        delta =  x - datetime_t if x > datetime_t else timedelta.max
        return delta
    
    #find the closest available radar to your datetime 
    
    closest_datetime = nearestDate(datetimes, datetime_t)
    index = datetimes.index(closest_datetime)
    #print(closest_datetime)
    #create a temp file, download radar data to file from S3
    #read into a radar object and return
    
    localfile = tempfile.NamedTemporaryFile(delete=False)
    keys[index].get_contents_to_filename(localfile.name)
    radar = pyart.io.read(localfile.name)
    name_str = str(bucket_list[index].key)
    return (radar,name_str[-22:-4])


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [4]:
#This cell does the heavy lifting with data and plotting. 
#Load shape files that contain county and urban area info 

cname='data//shapes//tl_2017_us_county.shp'

counties = ShapelyFeature(Reader(cname).geometries(),
                                ccrs.PlateCarree(), edgecolor='black',facecolor='none')

radar_name='KABR' #specify radar

#This block of code downloads radar data for 4 different times, then applies filters 
# to remove non-meteorlogical echo
b_d = datetime(2022,12,13,14,0) #datetime.utcnow() #datetime.strptime(base_date, fmt)
my_radar1 = get_radar_from_aws(radar_name,b_d)
gatefilter1 = pyart.filters.GateFilter(my_radar1[0])
gatefilter1.exclude_below('cross_correlation_ratio', 0.9)
gatefilter1.exclude_below('reflectivity', 20)
b_d = datetime(2019,7,10,19,17)
my_radar2 = get_radar_from_aws(radar_name,b_d)
gatefilter2 = pyart.filters.GateFilter(my_radar2[0])
gatefilter2.exclude_below('cross_correlation_ratio', 0.9)
gatefilter2.exclude_below('reflectivity', 20)
b_d = datetime(2019,7,10,19,36)
my_radar3 = get_radar_from_aws(radar_name,b_d)
gatefilter3 = pyart.filters.GateFilter(my_radar3[0])
gatefilter3.exclude_below('cross_correlation_ratio', 0.9)
gatefilter3.exclude_below('reflectivity', 20)
b_d = datetime(2019,7,10,19,52)
my_radar4 = get_radar_from_aws(radar_name,b_d)
gatefilter4 = pyart.filters.GateFilter(my_radar4[0])
gatefilter4.exclude_below('cross_correlation_ratio', 0.9)
gatefilter4.exclude_below('reflectivity', 20)

center_lon=my_radar15[0].longitude['data'][0] #Grab center coordinates of radar
center_lat=my_radar15[0].latitude['data'][0]  #Grab center coordinates of radar
#center_lon=-85.7585
#center_lat=38.2527
#Define range around center coordinates to plot.  This is in degrees latitude/longitude
min_lat = center_lat-1
max_lat = center_lat+1
min_lon = center_lon-1
max_lon = center_lon+1
projection = ccrs.PlateCarree()
lons=[-85,-85.5,-86,-86.5,-87]
lats=[37,37.5,38,38.5,39]
display = pyart.graph.RadarMapDisplay(my_radar1[0])
fig = plt.figure(figsize=(10, 8),dpi=600)  #Create figure
ax1 = fig.add_subplot(221,projection=projection) 
display.plot_ppi_map('reflectivity', 0, vmin=25, vmax=65,
                     min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
                     resolution='10m', projection=projection,colorbar_flag=0,
                     fig=fig, lat_0=center_lat,lon_0=center_lon,title = '1846 UTC',
                     lat_lines=lats, lon_lines=lons,gatefilter=gatefilter1,raster=True)
display.plot_point(my_radar1[0].longitude['data'][0], my_radar1[0].latitude['data'][0])
display.plot_range_rings([15,30,45,60,75],col='blue')
ax1.add_feature(cfeature.STATES.with_scale('10m'), edgecolor='black',linewidth=4)
ax1.add_feature(counties,linewidth=1)

ax1.text(-86.2, 38.65, 'a)',fontsize=14, fontweight='bold',transform=projection)

display = pyart.graph.RadarMapDisplay(my_radar2[0])
ax2= fig.add_subplot(222,projection=projection) 
display.plot_ppi_map('reflectivity', 0, vmin=25, vmax=65,
                     min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
                     resolution='10m', projection=projection,colorbar_flag=0,
                     fig=fig, lat_0=center_lat,
                     lon_0=center_lon,title = '1917 UTC',
                     lat_lines=lats, lon_lines=lons,gatefilter=gatefilter2,raster=True)
display.plot_point(my_radar1[0].longitude['data'][0], my_radar1[0].latitude['data'][0])
display.plot_range_rings([15,30,45,60,75],col='blue')
ax2.add_feature(cfeature.STATES.with_scale('10m'), edgecolor='black',linewidth=4)
ax2.add_feature(counties,linewidth=1)

#display.plot_point(-97.08697021,47.92184258,color='black',symbol='*',label_text='Skycam')
ax2.text(-86.2, 38.65, 'b)',fontsize=14, fontweight='bold',transform=projection)
display = pyart.graph.RadarMapDisplay(my_radar3[0])
ax3= fig.add_subplot(223,projection=projection) 
display.plot_ppi_map('reflectivity', 0, vmin=25, vmax=65,
                     min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
                     resolution='10m', projection=projection,colorbar_flag=0,
                     fig=fig, lat_0=center_lat,
                     lon_0=center_lon,title = '1936 UTC',
                     lat_lines=lats, lon_lines=lons,gatefilter=gatefilter3,raster=True)
display.plot_point(my_radar1[0].longitude['data'][0], my_radar1[0].latitude['data'][0])
display.plot_range_rings([15,30,45,60,75],col='blue')
ax3.add_feature(cfeature.STATES.with_scale('10m'), edgecolor='black',linewidth=4)
ax3.add_feature(counties,linewidth=1)

#display.plot_point(-97.08697021,47.92184258,color='black',symbol='*',label_text='Skycam')
ax3.text(-86.2, 38.65, 'c)',fontsize=14, fontweight='bold',transform=projection)
display = pyart.graph.RadarMapDisplay(my_radar4[0])
ax4= fig.add_subplot(224,projection=projection) 
display.plot_ppi_map('reflectivity', 0, vmin=25, vmax=65,
                     min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
                     resolution='10m', projection=projection,colorbar_flag=0,
                     fig=fig, lat_0=center_lat,
                     lon_0=center_lon,title = '1952 UTC',
                     lat_lines=lats, lon_lines=lons,gatefilter=gatefilter4,raster=True)
display.plot_point(my_radar1[0].longitude['data'][0], my_radar1[0].latitude['data'][0])
display.plot_range_rings([15,30,45,60,75],col='blue')
ax4.add_feature(cfeature.STATES.with_scale('10m'), edgecolor='black',linewidth=4)
ax4.add_feature(counties,linewidth=1)

#display.plot_point(-97.08697021,47.92184258,color='black',symbol='*',label_text='Skycam')
ax4.text(-86.2, 38.65, 'd)',fontsize=14, fontweight='bold',transform=projection)
#Setup colorbar
colorbar_panel_axes = [0.15, 0.04, 0.725, 0.05]
cbax = fig.add_axes(colorbar_panel_axes)
display.plot_colorbar(cax=cbax,orient='horizontal', label='Equivalent Reflectivity Factor (dBZ)')

plt.subplots_adjust(hspace = 0.25, wspace = 0.01)
plt.savefig('output//exam_winter_fig.png',dpi=300,bbox_inches='tight') #saves file to image
plt.show()


NameError: name 'my_radar15' is not defined

In [ ]:
print(center_lat)